In [41]:
import time
import multiprocessing
from multiprocessing import Pool

### Pool (map and reduce: divide the task and aggregate after finish)

In [45]:
def f(n):
    sum=0
    for x in range(1000):
        sum+=x*x
    return sum

In [51]:
if __name__=='__main__':
    t1=time.time()
    p=Pool(processes=12)
    result = p.map(f,range(100000))
    p.close()
    p.join()
    print('Pool took: ', time.time()-t1)
    
    t2=time.time()
    result=[]
    for x in range(100000):
        result.append(f(x))
    print('Serial Processing took: ', time.time()-t2)

Pool took:  2.861501932144165
Serial Processing took:  10.514475107192993


### normal multiple processing

In [52]:
def calc_square(numbers):
    for n in numbers:
        print('square ' + str(n*n))

def calc_cube(numbers):
    for n in numbers:
        print('cube ' + str(n*n*n))

if __name__ == "__main__":
    arr = [2,3,8]
    p1 = multiprocessing.Process(target=calc_square, args=(arr,))
    p2 = multiprocessing.Process(target=calc_cube, args=(arr,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print("Done!")

square 4
square 9
cube 8
square 64
cube 27
cube 512
Done!


### threading vs processing

In [ ]:
import time
import multiprocessing

In [5]:
square_result=[]
def calc_square(numbers):
    global square_result
    for n in numbers:
        print('square ' + str(n*n))
        square_result.append(n*n)
if __name__=='__main__':
    arr=[2,3,8,9]
    p1=multiprocessing.Process(target=calc_square,args=(arr,))
    p1.start()
    p1.join()
    print('outside result' + str(square_result))
    print('Done!')

square 4
square 9
square 64
square 81
outside result[]
Done!


**Comment**: 
When create a new process what will hapeen is it will create a new copy of square_result, global variable.
**Every process has its own address space.** Thus program vaiables are not shared between two processes.
**This is the key difference between multi-threading and multi-processing.**

In [6]:
square_result=[]
def calc_square(numbers):
    global square_result
    for n in numbers:
        print('square ' + str(n*n))
        square_result.append(n*n)
    print('within a process: result ', square_result)
if __name__=='__main__':
    arr=[2,3,8,9]
    p1=multiprocessing.Process(target=calc_square,args=(arr,))
    p1.start()
    p1.join()
    print('outside result' + str(square_result)) # this is outside the process
    print('Done!')

square 4
square 9
square 64
square 81
within a process: result  [4, 9, 64, 81]
outside result[]
Done!


In [7]:
# or use queue
import multiprocessing

def calc_square(numbers, q):
    for n in numbers:
        q.put(n*n)

if __name__ == "__main__":
    numbers = [2,3,5]
    q = multiprocessing.Queue()
    p = multiprocessing.Process(target=calc_square, args=(numbers,q))

    p.start()
    p.join()

    while q.empty() is False:
        print(q.get())

4
9
25


In [ ]:
multiprocessing queue
import multiprocessing
q=multiprocessing.Queue()
. lives in shared memory
. used to share data between processes

queue module
import queue
q=queue.Queue()
.live in in-process memory
. used to share data between threads

### lock

In [2]:
def deposit(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value = balance.value + 1
        lock.release()

def withdraw(balance, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value = balance.value - 1
        lock.release()

In [3]:
if __name__ == '__main__':
    balance = multiprocessing.Value('i', 200) # share between different processes !!!
    lock = multiprocessing.Lock()
    d = multiprocessing.Process(target=deposit, args=(balance,lock))
    w = multiprocessing.Process(target=withdraw, args=(balance,lock))
    d.start()
    w.start()
    d.join()
    w.join()
    print(balance.value)

200


### If without using lock, the result will be different each time !!!

In [40]:
def deposit(balance):
    for i in range(100):
        time.sleep(0.01)
        balance.value = balance.value + 1

def withdraw(balance):
    for i in range(100):
        time.sleep(0.01)
        balance.value = balance.value - 1

if __name__ == '__main__':
    balance = multiprocessing.Value('i', 200)
    d = multiprocessing.Process(target=deposit, args=(balance,))
    w = multiprocessing.Process(target=withdraw, args=(balance,))
    d.start()
    w.start()
    d.join()
    w.join()
    print(balance.value)

194


### multiple threading

In [57]:
import time
import threading

def calc_square(numbers):
    print("calculate square numbers")
    for n in numbers:
        time.sleep(1)
        print('square:',n*n)

def calc_cube(numbers):
    print("calculate cube of numbers")
    for n in numbers:
        time.sleep(1)
        print('cube:',n*n*n)

arr = [2,3,8,9]

t = time.time()

t1= threading.Thread(target=calc_square, args=(arr,))
t2= threading.Thread(target=calc_cube, args=(arr,))

t1.start()
t2.start()

t1.join()
t2.join()

print("done in : ",time.time()-t)
print("Hah... I am done with all my work now!")

calculate square numberscalculate cube of numbers

cube:square: 8
 4
square:cube: 9
 27
cube:square: 512
 64
cube:square: 729
 81
done in :  4.0178937911987305
Hah... I am done with all my work now!
